<a href="https://colab.research.google.com/github/CoGian/pegasus_demo_huggingface/blob/master/pegasus_demo_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the master version of huggingface's tranformers from github

In [1]:
!git clone https://github.com/huggingface/transformers
!pip install transformers/.

fatal: destination path 'transformers' already exists and is not an empty directory.
Processing ./transformers
  Created wheel for transformers: filename=transformers-3.1.0-cp36-none-any.whl size=884119 sha256=3f2ae51f32831ce9f4f4453780dfcaa1d995f63154e9923cedb74440b8101c5e
  Stored in directory: /tmp/pip-ephem-wheel-cache-w15x654z/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Found existing installation: transformers 3.1.0
    Uninstalling transformers-3.1.0:
      Successfully uninstalled transformers-3.1.0


In [2]:
!pip install rouge
from rouge import Rouge  
rouge = Rouge()

In [3]:
with open('article.txt','r') as f:
    body = f.read()

# Pegasus

In [4]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
class PegasusDemo():

    def __init__(self, model_name):
        self.torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = PegasusTokenizer.from_pretrained(model_name)
        self.model = PegasusForConditionalGeneration.from_pretrained(model_name).to(self.torch_device)
        
    def make_summary(self, body):
        batch = self.tokenizer.prepare_seq2seq_batch([body], truncation=True, padding='longest').to(self.torch_device)
        translated = self.model.generate(**batch)
        tgt_text = self.tokenizer.batch_decode(translated, skip_special_tokens=True)
        return tgt_text[0]


In [5]:
pgd = PegasusDemo('google/pegasus-xsum')
# more models in https://huggingface.co/models?search=pegasus

In [6]:
summary = pgd.make_summary(body)

In [7]:
summary

'Russia is facing renewed scrutiny for its cyber espionage efforts after the U.S., Great Britain and Canada alleged Thursday that a Kremlin-linked hacking group is attempting to steal research related to coronavirus vaccine developments and testing.'

In [8]:
scores = rouge.get_scores(summary, body)
print(scores)

[{'rouge-1': {'f': 0.038364462016445486, 'p': 1.0, 'r': 0.0195573854863613}, 'rouge-2': {'f': 0.037392622169699484, 'p': 1.0, 'r': 0.01905252317198764}, 'rouge-l': {'f': 0.11666666556805556, 'p': 1.0, 'r': 0.061946902654867256}}]
